In [ ]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_idx = test['key']
train_len = train.shape[0]

In [ ]:
df_all = pd.concat([train,test])
df_all = pd.get_dummies(df_all)

In [ ]:
apartment = df_all.groupby(['apartment_id'])['latitude','longitude','address_by_law'].mean().reset_index()
apartment.head()

In [ ]:
# 위도, 경도 거리 계산식
from math import cos, asin, sqrt
# 단위는 km로 나옴.
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

## 지하철
- 반경 0.5, 1km 이내의 지하철 갯수
- 반경 0.5, 1km 이내의 지하철 호선 수

### 서울 

In [ ]:
subway = pd.read_csv("Subways.csv")
subway_seoul = subway[subway['latitude']//1 == 37].reset_index(drop=True)
subway_busan = subway[subway['latitude']//1 == 35].reset_index(drop=True)
apartment_seoul = apartment[apartment['latitude']//1 == 37].reset_index(drop=True)
apartment_busan = apartment[apartment['latitude']//1 == 35].reset_index(drop=True)

In [ ]:
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
subway_lat_lon = subway_seoul[['latitude','longitude']].reset_index(drop=True)

In [ ]:
#서울 지하철 갯수 구하기
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0
        count2 = 0
        count3 = 0
        count4 = 0
        count5 = 0
        count6 = 0
        count7 = 0
        count8 = 0
        count9 = 0
        countKJ = 0
        countAP = 0
        countUS = 0
        countKC = 0
        countDL = 0
        countND = 0
        countall = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 0.5:
                    if '1' in subway_seoul['subway_line'][j]:
                        count1 = 1
                    if '2' in subway_seoul['subway_line'][j]:
                        count2 = 1
                    if '3' in subway_seoul['subway_line'][j] :
                        count3 = 1
                    if '4' in subway_seoul['subway_line'][j] :
                        count4 = 1
                    if '5' in subway_seoul['subway_line'][j] :
                        count5 = 1
                    if '6' in subway_seoul['subway_line'][j]:
                        count6 = 1
                    if '7' in subway_seoul['subway_line'][j] :
                        count7 = 1
                    if '8' in subway_seoul['subway_line'][j] :
                        count8 = 1
                    if '9' in subway_seoul['subway_line'][j] :
                        count9 = 1
                    if 'KJ' in subway_seoul['subway_line'][j] :
                        countKJ = 1
                    if 'AP' in subway_seoul['subway_line'][j] :
                        countAP = 1
                    if 'US' in subway_seoul['subway_line'][j] :
                        countUS = 1
                    if 'KC' in subway_seoul['subway_line'][j] :
                        countKC = 1
                    if 'DL' in subway_seoul['subway_line'][j] :
                        countDL = 1
                    if 'ND' in subway_seoul['subway_line'][j] :
                        countND = 1
                    else:
                        countall = 0
                else:
                    countall = 0
                    
        countall = count1 + count2 + count3 + count4 + count5 + count6 + count7 + count8 + count9 + countKJ + countAP + countUS + countKC + countDL + countND 
        count_list.append(countall)
apartment_seoul['subwayline_count_0.5'] = count_list

In [ ]:
#서울 지하철 갯수 구하기
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0
        count2 = 0
        count3 = 0
        count4 = 0
        count5 = 0
        count6 = 0
        count7 = 0
        count8 = 0
        count9 = 0
        countKJ = 0
        countAP = 0
        countUS = 0
        countKC = 0
        countDL = 0
        countND = 0
        countall = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    if '1' in subway_seoul['subway_line'][j]:
                        count1 = 1
                    if '2' in subway_seoul['subway_line'][j]:
                        count2 = 1
                    if '3' in subway_seoul['subway_line'][j] :
                        count3 = 1
                    if '4' in subway_seoul['subway_line'][j] :
                        count4 = 1
                    if '5' in subway_seoul['subway_line'][j] :
                        count5 = 1
                    if '6' in subway_seoul['subway_line'][j]:
                        count6 = 1
                    if '7' in subway_seoul['subway_line'][j] :
                        count7 = 1
                    if '8' in subway_seoul['subway_line'][j] :
                        count8 = 1
                    if '9' in subway_seoul['subway_line'][j] :
                        count9 = 1
                    if 'KJ' in subway_seoul['subway_line'][j] :
                        countKJ = 1
                    if 'AP' in subway_seoul['subway_line'][j] :
                        countAP = 1
                    if 'US' in subway_seoul['subway_line'][j] :
                        countUS = 1
                    if 'KC' in subway_seoul['subway_line'][j] :
                        countKC = 1
                    if 'DL' in subway_seoul['subway_line'][j] :
                        countDL = 1
                    if 'ND' in subway_seoul['subway_line'][j] :
                        countND = 1
                    else:
                        countall = 0
                else:
                    countall = 0
                    
        countall = count1 + count2 + count3 + count4 + count5 + count6 + count7 + count8 + count9 + countKJ + countAP + countUS + countKC + countDL + countND 
        count_list.append(countall)
apartment_seoul['subwayline_count_1'] = count_list

In [ ]:
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 0.5:
                    count = count + 1
        count_list.append(count)
apartment_seoul['subway_count_0.5'] = count_list

In [ ]:
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    count = count + 1
        count_list.append(count)
apartment_seoul['subway_count_1'] = count_list

### 부산 

In [ ]:
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)
subway_lat_lon = subway_busan[['latitude','longitude']].reset_index(drop=True)

In [ ]:
#부산 지하철 갯수 구하기
# BK, B1,B2,B3 ,B4, BD
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        countBK = 0
        countB1 = 0
        countB2 = 0
        countB3 = 0
        countB4 = 0
        countBD = 0
        countall = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 0.5:
                    if 'B1' in subway_busan['subway_line'][j]:
                        countB1 = 1
                    if 'B2' in subway_busan['subway_line'][j]:
                        countB2 = 1
                    if 'B3' in subway_busan['subway_line'][j] :
                        countB3 = 1
                    if 'B4' in subway_busan['subway_line'][j] :
                        countB4 = 1
                    if 'BK' in subway_busan['subway_line'][j] :
                        countBK = 1
                    if 'BD' in subway_busan['subway_line'][j]:
                        countBD = 1
                    else:
                        countall = 0
                else:
                    countall = 0
                    
        countall = countB1 + countB2 + countB3 + countB4 + countBD + countBK 
        count_list.append(countall)
apartment_busan['subwayline_count_0.5'] = count_list

In [ ]:
#부산 지하철 갯수 구하기
# BK, B1,B2,B3 ,B4, BD
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        countBK = 0
        countB1 = 0
        countB2 = 0
        countB3 = 0
        countB4 = 0
        countBD = 0
        countall = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    if 'B1' in subway_busan['subway_line'][j]:
                        countB1 = 1
                    if 'B2' in subway_busan['subway_line'][j]:
                        countB2 = 1
                    if 'B3' in subway_busan['subway_line'][j] :
                        countB3 = 1
                    if 'B4' in subway_busan['subway_line'][j] :
                        countB4 = 1
                    if 'BK' in subway_busan['subway_line'][j] :
                        countBK = 1
                    if 'BD' in subway_busan['subway_line'][j]:
                        countBD = 1
                    else:
                        countall = 0
                else:
                    countall = 0
                    
        countall = countB1 + countB2 + countB3 + countB4 + countBD + countBK 
        count_list.append(countall)
apartment_busan['subwayline_count_1'] = count_list

In [ ]:
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 0.5:
                    count = count + 1
        count_list.append(count)
apartment_busan['subway_count_0.5'] = count_list

In [ ]:
count_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count = 0
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    count = count + 1
        count_list.append(count)
apartment_busan['subway_count_1'] = count_list

In [ ]:
apartment = pd.concat([apartment_seoul,apartment_busan])
apartment.to_csv("Apartment_subway.csv",index=False)

## 학교
- 반경 0.5km, 1km내의 초, 중, 고등학교의 수
- 반경 0.5km, 1km내에 초, 중, 고 중 몇개가 있는지 

In [ ]:
school = pd.read_csv("Schools.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
df_all = pd.concat([train,test])

In [ ]:
from math import cos, asin, sqrt
# 단위는 km로 나옴.
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [ ]:
school_seoul = school[school['latitude']//1 == 37].reset_index(drop=True)
school_busan = school[school['latitude']//1 == 35].reset_index(drop=True)
apartment_seoul = apartment[apartment['latitude']//1 == 37].reset_index(drop=True)
apartment_busan = apartment[apartment['latitude']//1 == 35].reset_index(drop=True)

In [ ]:
#서울 학교갯수 0.5km
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
school_seoul_lat_lon = school_seoul[['latitude','longitude']].reset_index(drop=True)

count_list1 = []
count_list2 = []
count_list3 = []

for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0
        count2 = 0
        count3 = 0

        for j in range(0,school_seoul_lat_lon.shape[0]):
            for lat2,lon2 in [school_seoul_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 0.5:
                    if school_seoul['school_class'][j] == 'elementary':
                        count1 = count1+1

                    elif school_seoul['school_class'][j] == 'middle':
                        count2 = count2+1
                    else :
                        count3 = count3+1
                    
        count_list1.append(count1)
        count_list2.append(count2)
        count_list3.append(count3)
apartment_seoul['elementary_0.5'] = count_list1
apartment_seoul['middle_0.5'] = count_list2
apartment_seoul['high_0.5'] = count_list3

In [ ]:
#서울 학교갯수 1km
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
school_seoul_lat_lon = school_seoul[['latitude','longitude']].reset_index(drop=True)

count_list1 = []
count_list2 = []
count_list3 = []

for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0
        count2 = 0
        count3 = 0

        for j in range(0,school_seoul_lat_lon.shape[0]):
            for lat2,lon2 in [school_seoul_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    if school_seoul['school_class'][j] == 'elementary':
                        count1 = count1+1 

                    elif school_seoul['school_class'][j] == 'middle':
                        count2 = count2+1
                    else :
                        count3 = count3+1
                    
        count_list1.append(count1)
        count_list2.append(count2)
        count_list3.append(count3)
apartment_seoul['elementary_1'] = count_list1
apartment_seoul['middle_1'] = count_list2
apartment_seoul['high_1'] = count_list3

In [ ]:
#부산학교갯수 0.5km
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)
school_busan_lat_lon = school_busan[['latitude','longitude']].reset_index(drop=True)

count_list1 = []
count_list2 = []
count_list3 = []

for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0
        count2 = 0
        count3 = 0

        for j in range(0,school_busan_lat_lon.shape[0]):
            for lat2,lon2 in [school_busan_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 0.5:
                    if school_busan['school_class'][j] == 'elementary':
                        count1 = count1+1

                    elif school_busan['school_class'][j] == 'middle':
                        count2 = count2+1
                    else :
                        count3 = count3+1
                    
        count_list1.append(count1)
        count_list2.append(count2)
        count_list3.append(count3)
apartment_busan['elementary_0.5'] = count_list1
apartment_busan['middle_0.5'] = count_list2
apartment_busan['high_0.5'] = count_list3

In [ ]:
#부산 학교갯수 1km
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)
school_busan_lat_lon = school_busan[['latitude','longitude']].reset_index(drop=True)

count_list1 = []
count_list2 = []
count_list3 = []

for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0
        count2 = 0
        count3 = 0

        for j in range(0,school_busan_lat_lon.shape[0]):
            for lat2,lon2 in [school_busan_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    if school_busan['school_class'][j] == 'elementary':
                        count1 = count1+1

                    elif school_busan['school_class'][j] == 'middle':
                        count2 = count2+1

                    else :
                        count3 = count3+1
                    
        count_list1.append(count1)
        count_list2.append(count2)
        count_list3.append(count3)
apartment_busan['elementary_1'] = count_list1
apartment_busan['middle_1'] = count_list2
apartment_busan['high_1'] = count_list3

In [ ]:
apartment_seoul['total_0.5'] = apartment_seoul['elementary_0.5'] + apartment_seoul['middle_0.5'] + apartment_seoul['high_0.5']
apartment_busan['total_0.5'] = apartment_busan['elementary_0.5'] + apartment_busan['middle_0.5'] + apartment_busan['high_0.5']
apartment_seoul['total_1'] = apartment_seoul['elementary_1'] + apartment_seoul['middle_1'] + apartment_seoul['high_1']
apartment_busan['total_1'] = apartment_busan['elementary_1'] + apartment_busan['middle_1'] + apartment_busan['high_1']

In [ ]:
apartment = pd.concat([apartment_seoul,apartment_busan])
apartment.head(5)

In [ ]:
apartment['elementary_0.5'] = apartment['elementary_0.5'].apply(lambda x : 1 if x >= 1 else 0)
apartment['middle_0.5'] = apartment['middle_0.5'].apply(lambda x : 1 if x >= 1 else 0)
apartment['high_0.5'] = apartment['high_0.5'].apply(lambda x : 1 if x >= 1 else 0)
apartment['total_0.5'] = apartment['elementary_0.5'] + apartment['middle_0.5'] + apartment['high_0.5']
apartment['elementary_1'] = apartment['elementary_1'].apply(lambda x : 1 if x >= 1 else 0)
apartment['middle_1'] = apartment['middle_1'].apply(lambda x : 1 if x >= 1 else 0)
apartment['high_1'] = apartment['high_1'].apply(lambda x : 1 if x >= 1 else 0)
apartment['total_1'] = apartment['elementary_1'] + apartment['middle_1'] + apartment['high_1']

In [ ]:
apartment.to_csv("apartment_school.csv",index=False)

## 지하철 및 학교에서 최소거리

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_idx = test['key']
train_len = train.shape[0]

In [ ]:
df_all = pd.concat([train,test])
apartment = df_all.groupby(['apartment_id'])['latitude','longitude','address_by_law'].mean().reset_index()

In [ ]:
subway = pd.read_csv("Subways.csv")
subway.head()

In [ ]:
from math import cos, asin, sqrt
# 단위는 km로 나옴.
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

### 지하철에서 최소거리 계산

In [ ]:
subway_seoul = subway[subway['latitude']//1 == 37].reset_index(drop=True)
subway_busan = subway[subway['latitude']//1 == 35].reset_index(drop=True)
apartment_seoul = apartment[apartment['latitude']//1 == 37].reset_index(drop=True)
apartment_busan = apartment[apartment['latitude']//1 == 35].reset_index(drop=True)

In [ ]:
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
subway_lat_lon = subway_seoul[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 서울-지하철에서 최소거리
distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_seoul['subway_min_distance'] = distance_list

In [ ]:
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)
subway_lat_lon = subway_busan[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 부산-지하철에서 최소거리
distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,subway_lat_lon.shape[0]):
            for lat2,lon2 in [subway_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_busan['subway_min_distance'] = distance_list

### 초,중,고에서 최소거리 계산

In [ ]:
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
school = pd.read_csv("Schools.csv")
school_seoul = school[school['latitude']//1 == 37].reset_index(drop=True)
school_busan = school[school['latitude']//1 == 35].reset_index(drop=True)

서울

In [ ]:
school_seoul_ele = school_seoul[school_seoul['school_class']=='elementary']
school_seoul_lat_lon = school_seoul_ele[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 서울-초등학교 최소거리

distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,school_seoul_lat_lon.shape[0]):
            for lat2,lon2 in [school_seoul_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_seoul['min_distance_ele'] = distance_list

In [ ]:
school_seoul_middle = school_seoul[school_seoul['school_class']=='middle']
school_seoul_lat_lon = school_seoul_middle[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 서울-중학교 최소거리

distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,school_seoul_lat_lon.shape[0]):
            for lat2,lon2 in [school_seoul_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_seoul['min_distance_middle'] = distance_list

In [ ]:
school_seoul_high = school_seoul[school_seoul['school_class']=='high']
school_seoul_lat_lon = school_seoul_high[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 서울-고등학교 최소거리

distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,school_seoul_lat_lon.shape[0]):
            for lat2,lon2 in [school_seoul_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_seoul['min_distance_high'] = distance_list

부산

In [ ]:
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)

In [ ]:
school_busan_ele = school_busan[school_busan['school_class']=='elementary']
school_busan_lat_lon = school_busan_ele[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 부산-초등학교 최소거리
distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,school_busan_lat_lon.shape[0]):
            for lat2,lon2 in [school_busan_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_busan['min_distance_ele'] = distance_list

In [ ]:
school_busan_middle = school_busan[school_busan['school_class']=='middle']
school_busan_lat_lon = school_busan_middle[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 부산-중학교 최소거리
distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,school_busan_lat_lon.shape[0]):
            for lat2,lon2 in [school_busan_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_busan['min_distance_middle'] = distance_list

In [ ]:
school_busan_high = school_busan[school_busan['school_class']=='high']
school_busan_lat_lon = school_busan_high[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 부산-고등학교 최소거리
distance_list = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        min_distance = 100000000
        for j in range(0,school_busan_lat_lon.shape[0]):
            for lat2,lon2 in [school_busan_lat_lon.loc[j].values]:
                distance1 = distance(lat1,lon1,lat2,lon2)
                if distance1 < min_distance:
                    min_distance = distance1
        distance_list.append(min_distance)
apartment_busan['min_distance_high'] = distance_list

In [ ]:
apartment = pd.concat([apartment_seoul,apartment_busan])
apartment.to_csv("min_distance_apartment.csv",index=False)

## 강남 및 해운대에서 최소거리

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_idx = test['key']
train_len = train.shape[0]

df_all = pd.concat([train,test])
apartment = df_all.groupby(['apartment_id'])['latitude','longitude','address_by_law'].mean().reset_index()

In [ ]:
subway = pd.read_csv("Subways.csv")
#강남역 위도/경도 37.4979312,127.0232248
#해운대역 위도/경도 35.1816217,129.1751215

In [ ]:
from math import cos, asin, sqrt
# 단위는 km로 나옴.
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [ ]:
subway_seoul = subway[subway['latitude']//1 == 37].reset_index(drop=True)
subway_busan = subway[subway['latitude']//1 == 35].reset_index(drop=True)
apartment_seoul = apartment[apartment['latitude']//1 == 37].reset_index(drop=True)
apartment_busan = apartment[apartment['latitude']//1 == 35].reset_index(drop=True)

In [ ]:
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 강남역에서 거리 
gangnam_dist = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        gangnam_dist.append(distance(lat1,lon1,37.4979312,127.0232248))
apartment_seoul['gangnam_dist'] = gangnam_dist

In [ ]:
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)

In [ ]:
### 해운대역에서 거리 - 편의상 변수명은 gangnam으로 지정.
hyeunde_dist = []
for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        hyeunde_dist.append(distance(lat1,lon1,35.1816217,129.1751215))
apartment_busan['gangnam_dist'] = hyeunde_dist

In [ ]:
apartment = pd.concat([apartment_seoul,apartment_busan])
apartment.to_csv("Apartment_ga_hd.csv",index=False)